資料的突出值要去掉

In [16]:
import pandas as pd
import numpy as np

In [17]:
data = pd.read_csv("./Data/Taipei/h23-g1.csv",delimiter=';')

In [18]:
print(data)

                 date_time            E             N         H  Angle  Axis  \
0      2023-03-22 12:10:00  320191.8741  2.771896e+06  287.4545    0.0   0.0   
1      2023-03-22 12:20:00  320191.8742  2.771896e+06  287.4555    0.0   0.0   
2      2023-03-22 12:30:00  320191.8749  2.771896e+06  287.4554    0.0   0.0   
3      2023-03-22 12:40:00  320191.8757  2.771896e+06  287.4576    0.0   0.0   
4      2023-03-22 12:50:00  320191.8771  2.771896e+06  287.4603    0.0   0.0   
...                    ...          ...           ...       ...    ...   ...   
74028  2024-12-15 10:40:00  320191.8353  2.771896e+06  287.4683   35.0   5.4   
74029  2024-12-15 10:50:00  320191.8353  2.771896e+06  287.4683   36.0   5.2   
74030  2024-12-15 11:00:00  320191.8353  2.771896e+06  287.4683   41.0   6.2   
74031  2024-12-15 11:10:00  320191.8349  2.771896e+06  287.4669   39.0   5.2   
74032  2024-12-15 11:20:00  320191.8341  2.771896e+06  287.4743   65.0   9.3   

       Plate  EMove  NMove  HMove  Tota

In [19]:
data['date_time'] = pd.to_datetime(data['date_time'])
data['date_time'] = data['date_time'].astype(int) / (10 ** 11)

In [20]:
print(data['date_time'])

0        16794870.0
1        16794876.0
2        16794882.0
3        16794888.0
4        16794894.0
            ...    
74028    17342592.0
74029    17342598.0
74030    17342604.0
74031    17342610.0
74032    17342616.0
Name: date_time, Length: 74033, dtype: float64


In [21]:
# Remove values outside 3 standard deviations
target = ['EMove', 'NMove', 'HMove']

for col in target:
    mean = data[col].mean()
    std = data[col].std()
    data = data[(data[col] >= mean - 3*std) & (data[col] <= mean + 3*std)]

In [22]:
lag_features = ['EMove', 'NMove', 'HMove']
lag_steps = 3

for feature in lag_features:
    for lag in range(1, lag_steps + 1):
        data[f'{feature}_lag{lag}'] = data[feature].shift(lag)

data = data.dropna()

In [23]:
features = ["date_time"] + [f"{feature}_lag1" for feature in lag_features] + [f"{feature}_lag2" for feature in lag_features] + [f"{feature}_lag3" for feature in lag_features]

X = data[features]
y = data[target]

In [24]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
xgbrModel=xgb.XGBRegressor(
    n_estimators=300,      # 樹的數量
    max_depth=8,           # 樹的最大深度
    learning_rate=0.05,    # 學習率
    subsample=0.8,         # 訓練數據取樣比例
    colsample_bytree=0.8,  # 特徵取樣比例
    reg_lambda=10,          # L2 正則化
    reg_alpha=1,         # L1 正則化
    min_child_weight=1,    # 限制葉子節點最小權重
    gamma=0.1,             # 節點劃分限制
    random_state=42)
xgbrModel.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [27]:
y_pred = xgbrModel.predict(X_test)

In [28]:
print(y_pred)

[[ -3.5038545    8.441495     1.1730131 ]
 [ -0.55880386   4.995192    -2.458065  ]
 [-22.88608     33.998558    -0.46601212]
 ...
 [-44.965187    63.71041      8.157626  ]
 [ -1.6751664    7.045397    20.873207  ]
 [-21.205473    35.78887      4.8407154 ]]


In [29]:
from sklearn.metrics import mean_squared_error,mean_absolute_error

E_mae = mean_absolute_error(y_test['EMove'], y_pred[:,0])
E_rmse = np.sqrt(mean_squared_error(y_test['EMove'], y_pred[:,0]))

N_mae = mean_absolute_error(y_test['NMove'], y_pred[:,1])
N_rmse = np.sqrt(mean_squared_error(y_test['NMove'], y_pred[:,1]))

H_mae = mean_absolute_error(y_test['HMove'], y_pred[:,2])
H_rmse = np.sqrt(mean_squared_error(y_test['HMove'], y_pred[:,2]))

#TotalM_mae = mean_absolute_error(y_test['TotalMove'], y_pred[:,3])
#TotalM_rmse = np.sqrt(mean_squared_error(y_test['TotalMove'], y_pred[:,3]))

results = pd.DataFrame({
    "Variable": ["EMove","NMove","HMove"],
    "MAE": [E_mae,N_mae,H_mae],
    "RMSE": [E_rmse,N_rmse,H_rmse]
})

print("results:")
print(results)

results:
  Variable       MAE      RMSE
0    EMove  0.955842  1.570090
1    NMove  1.021267  1.623595
2    HMove  2.825417  4.974141


In [30]:
for t in target:
    y_min = np.min(y_test[t], axis=0)  # 最小值
    y_max = np.max(y_test[t], axis=0)  # 最大值
    y_range = y_max - y_min
    print(t,"range:",y_range,'\n')
    y_std = np.std(y_test[t], axis=0) 
    print(t,"std:",y_std,'\n')

EMove range: 75.3 

EMove std: 11.15129655491409 

NMove range: 91.5 

NMove std: 15.989297469346242 

HMove range: 102.7 

HMove std: 12.59635304087366 

